- https://zenn.dev/tishii2479/articles/6b381fb86e0369

# 共通部分

In [1]:
import requests
import json
import os
import re
import html
import chromedriver_binary
from selenium import webdriver
from time import sleep
import subprocess

userID = "darancho" # 自分のAtCoderのユーザーIDを設定する
api_path = "https://kenkoooo.com/atcoder/atcoder-api/results?user="

# ディレクトリ作成
root = "/Users/nakamurataichi/pythonwork/github/atcoder/"

# APIを用いた提出データの取得
def getSubmissionData(userID):
    api_url = api_path + userID
    response = requests.get(api_url)
    jsonData = response.json()
    return jsonData

# 各問題において最も新しいAC提出のみを取得する
# 各コンテストごとにまとめて返す
def collectNewestAcceptedSubmissions(submissions):
    sortedData = sorted(submissions, key=lambda x: x['id'])  # IDで昇順ソートすると古い順になる
    submits = {} # 各問題ごとに最新の提出に更新する

    for data in sortedData:
        if data["result"] == "AC" and "Py" in data["language"]: # ACだった提出だけ対象かつpythonのみ
            submits[data["problem_id"]] = data

    result = {} # コンテストごとにまとめる
    for sub in submits.values():
        if not sub["contest_id"] in result:
            result[sub["contest_id"]] = []
        result[sub["contest_id"]].append(sub)
    return result

submissions = getSubmissionData(userID)
acSubmits =  collectNewestAcceptedSubmissions(submissions)

# コード登録

In [ ]:
for contestName in acSubmits:
    path = root + contestName
    os.makedirs(path, exist_ok=True)

In [ ]:
driver = webdriver.Chrome()

# 追加したファイルの数を増やす
add_cnt = 0

for submissions in acSubmits.values():
    for sub in submissions:
        # 問題番号の取得
        problem_num = sub["problem_id"].split("_")[-1]
        
        if problem_num.isdigit():
            problem_num = chr(int(problem_num)+ord('a')-1)
        
        # 作成するファイルへのパス
        path = root + sub["contest_id"] + "/" + problem_num
        
        # 古い問題の場合には数字になっているので、アルファベットに戻す
        if problem_num.isdigit():
            problem_num = chr(int(problem_num)+ord('a')-1)
        
        # 作成するファイルへのパス
        path = root + sub["contest_id"] + "/" + problem_num
        # 拡張子の設定（C++, Pythonのみ）
        if "C++" in sub["language"]:
            path += ".cpp"
        elif "Python" in sub["language"]:
            path += ".py"
        
        # 既に提出コードがある場合は取得せず、次の問題の提出を探す
        if os.path.isfile(path): continue
            
        # 提出ページへアクセス
        sub_url = "https://atcoder.jp/contests/" + sub["contest_id"] + "/submissions/" + str(sub["id"])
        
        driver.get(sub_url)
        
        # 提出コードの取得
        code = driver.find_element_by_id("submission-code") 
        
        # code.text は提出時に含めていない空白が期待に反して含まれてしまう
        # 空白はシンタックスハイライティングによるものであるように見える
        # innerHTML から不要なタグなどを消し、空白が意図通りのテキストを得る
        inner_html = code.get_attribute('innerHTML')
        list_items = re.findall(r'<li[^>]*>.*?</li>', inner_html)
        lines = []
        for li in list_items:
            line1 = re.sub(r'<[^>]+>', '', li)
            line2 = re.sub(r'&nbsp;', '', line1)
            line3 = html.unescape(line2)
            lines.append(line3 + "\n")
        code_text = ''.join(lines)
            
        # 書き込み
        with open(path, 'w') as f:
            f.write(code_text)
        
        # C++の場合にはclang-formatを使ってフォーマットする
        #if "C++" in sub["language"]:
         #   subprocess.call(["clang-format", "-i",  "-style=file", path])
        
        # 追加したファイルの数を増やす
        add_cnt += 1
            
        # アクセス負荷軽減のために時間をおく(3秒)
        sleep(3)
        
driver.quit()

# 解いた問題をリスト化

In [4]:
contest_data = {}
for submissions in acSubmits.values():
    problem = [] #
    contest_id = submissions[0]["contest_id"] #
    for sub in submissions:
        # 問題番号の取得
        problem_num = sub["problem_id"].split("_")[-1]
        # 古い問題の場合には数字になっているので、アルファベットに戻す
        if problem_num.isdigit():
            problem_num = chr(int(problem_num)+ord('a')-1)
        problem.append(problem_num) #
    contest_data[contest_id] = problem #

In [5]:
import pprint #辞書形式をきれいに出力するため
pprint.pprint(contest_data, compact=True)

{'abc042': ['a', 'b', 'a'],
 'abc043': ['a', 'b'],
 'abc044': ['a', 'b'],
 'abc045': ['a', 'b'],
 'abc046': ['a', 'b', 'a'],
 'abc047': ['a', 'b', 'a'],
 'abc048': ['a', 'b', 'a'],
 'abc049': ['a', 'b', 'a'],
 'abc050': ['a', 'b', 'a'],
 'abc051': ['a', 'b'],
 'abc052': ['a', 'b', 'a', 'b'],
 'abc053': ['a', 'b', 'a'],
 'abc054': ['a', 'b'],
 'abc055': ['a', 'b'],
 'abc056': ['a', 'b'],
 'abc057': ['a', 'b'],
 'abc058': ['a', 'b'],
 'abc059': ['a', 'b'],
 'abc060': ['a', 'b'],
 'abc061': ['a', 'b', 'c'],
 'abc062': ['a', 'b'],
 'abc063': ['a', 'b', 'a'],
 'abc064': ['a', 'b', 'c'],
 'abc065': ['a', 'b', 'a'],
 'abc066': ['a', 'b', 'a'],
 'abc067': ['a', 'b', 'a'],
 'abc068': ['a', 'b', 'a'],
 'abc069': ['a', 'b'],
 'abc070': ['a', 'b'],
 'abc071': ['a', 'b', 'a'],
 'abc072': ['a', 'b'],
 'abc073': ['a', 'b'],
 'abc074': ['a', 'b'],
 'abc075': ['a', 'b'],
 'abc076': ['a', 'b'],
 'abc077': ['a', 'b'],
 'abc078': ['a', 'b'],
 'abc079': ['a', 'b'],
 'abc080': ['b'],
 'abc081': ['a', 'b'],


In [4]:
import json
# https://qiita.com/Hyperion13fleet/items/7129623ab32bdcc6e203

# Serialize data into file:
json.dump(contest_data, open(root +"solved_contest_problem.json", 'w' ), ensure_ascii=False, indent=3, sort_keys=True, separators=(',', ': '))